In [1]:
# Step 1: Setup imports and paths
import sys
from pathlib import Path
sys.path.insert(0, str(Path().cwd() / "llm_qus_analyzer"))
import json
from llm_qus_analyzer import Settings, LLMClient, QUSChunkerModel
print("✅ Imports successful")

✅ Imports successful


In [2]:
# Step 2: Load settings and config
settings = Settings()
settings.configure_paths_and_load(
    env_path=Path(".env"),
    model_config_path=Path("models.yaml")
)
print("✅ Settings loaded successfully")
print(f"API key loaded: {settings.config.api_key[:10]}...")

✅ Settings loaded successfully
API key loaded: tgp_v1_9xj...


In [3]:
# Step 3: Initialize client and chunker
client = LLMClient(settings)
chunker = QUSChunkerModel()
print("✅ Client and chunker initialized")
print(f"Available models: {client.names}")

✅ Client and chunker initialized
Available models: ['Mixtral 8x7B']


In [ ]:
# import json

# # assumes: `client` and `chunker.analyze_single(client, idx, text)` already exist

# conflict_free = [
#   {
#     "pair": 1,
#     "stories": [
#       "As a User, I want to delete posts so that I remove spam.",
#       "As a User, I want to delete posts so that I hide content temporarily."
#     ],
#     "violation": "sameMeansDiffEnd"
#   },
#   {
#     "pair": 2,
#     "stories": [
#       "As a Shopper, I want to favorite products so that I find them later.",
#       "As a Shopper, I want to bookmark products so that I find them later."
#     ],
#     "violation": "diffMeansSameEnd"
#   },
#   {
#     "pair": 3,
#     "stories": [
#       "As a Driver, I want to pay tolls with cash so that I pass the gate.",
#       "As a Driver, I want to pass tolls without payment so that I pass the gate."
#     ],
#     "violation": "diffMeansSameEnd"
#   },
#   {
#     "pair": 4,
#     "stories": [
#       "As a Guest, I want to view all photos so that I enjoy the gallery.",
#       "As an Owner, I want Guests to view only public photos so that I protect privacy."
#     ],
#     "violation": "diffRoleSameStory"
#   },
#   {
#     "pair": 5,
#     "stories": [
#       "As a User, I want to edit profiles so that I update my information.",
#       "As a User, I want to edit profiles so that I change other users’ information."
#     ],
#     "violation": "sameMeansDiffEnd"
#   }
# ]

# results = []
# for item in user_stories:
#     pair_results = []
#     for story in item["stories"]:
#         component, usage = chunker.analyze_single(client, 0, story)
#         pair_results.append({
#             "user_story": story,
#             "expanded_text": component.text,
#             "role": component.role,
#             "means": component.means,
#             "ends": component.ends,
#             "template": {
#                 "text": component.template.text,
#                 "chunk": component.template.chunk,
#                 "tail": component.template.tail,
#                 "order": component.template.order
#             },
#             "usage": {
#                 "duration": usage.duration,
#                 "input_tokens": usage.num_token_in,
#                 "output_tokens": usage.num_token_out
#             }
#         })
#     results.append({
#         "pair": item["pair"],
#         "stories": pair_results
#     })

# try:
#     with open("landmark_stories.json", "w", encoding="utf-8") as f:
#         json.dump(results, f, indent=2, ensure_ascii=False)
# except OSError as e:
#     raise RuntimeError(f"Failed to write landmark_stories.json: {e.strerror}") from e


In [ ]:
with open("conflict_free_pair.json", "r") as f:
    data = json.load(f)


sameMeansDiffEnd
diffMeansSameEnd
diffMeansSameEnd
diffRoleSameStory
sameMeansDiffEnd


In [69]:
from typing import Any, Dict, List, Tuple

Pair = Dict[str, Any]
StoryCore = Tuple[List[str], str, str, str]  # (roles, means, ends, expanded_text)

def extract_core_zero_based(data: List[Pair]) -> Tuple[List[Dict[str, Any]], Dict[int, int]]:
    """
    Build a zero-based compact list of pairs.
    Returns (core_list, id_to_index):
      - core_list[i] = {"pair": pair_id, "violation": str, "stories": List[StoryCore]}
      - id_to_index[pair_id] = i
    """
    pairs_sorted = sorted(data, key=lambda p: p["pair"])
    core_list: List[Dict[str, Any]] = []

    for p in pairs_sorted:
        stories: List[StoryCore] = [
            (s["role"], s["means"], s["ends"], s["expanded_text"])
            for s in p["stories"]
        ]
        core_list.append({
            "pair": p["pair"],
            "violation": p.get("violation"),
            "stories": stories,
        })

    id_to_index = {entry["pair"]: idx for idx, entry in enumerate(core_list)}
    return core_list, id_to_index

# ---- usage ----
# data = [...]  # your full list from the message
core, idx = extract_core_zero_based(data)

# index 0 is the first pair
roles, means, ends, us = core[0]["stories"][0]

# access by pair id directly
pid = 3
i = idx[pid]
roles2, means2, ends2, us2 = core[i]["stories"][0]

# iterate all expanded_text
for entry in core:
    pid = entry["pair"]
    for _, _, _, u in entry["stories"]:
        print(f"[pair {pid}] {u}")


[pair 1] As a User, I want to delete posts so that I remove spam.
[pair 1] As a User, I want to delete posts so that I hide content temporarily.
[pair 2] As a shopper, I am able to favorite products so that I can find them later.
[pair 2] As a shopper, I am booking mark products so that I can find them later.
[pair 3] As a driver, I am paying tolls with cash so that I pass the gate.
[pair 3] As a driver, I am able to pass tolls without payment so that I pass the gate.
[pair 4] As a guest, I am viewing all photos so that I enjoy the gallery.
[pair 4] As an owner, I am able to ensure that guests view only public photos so that I protect privacy.
[pair 5] As a user, I am editing profiles so that I update my information.
[pair 5] As a user, I am editing profiles so that I can change other users’ information.


{'violation': None, 'stories': []}

In [72]:
core[0]

{'pair': 1,
 'violation': 'sameMeansDiffEnd',
 'stories': [(['User'],
   'delete posts',
   'remove spam',
   'As a User, I want to delete posts so that I remove spam.'),
  (['User'],
   'delete posts',
   'hide content temporarily',
   'As a User, I want to delete posts so that I hide content temporarily.')]}